# Installing dependancies

In [1]:
!pip install datasets accelerate --quiet

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
import re

2024-06-10 15:00:56.797706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 15:00:56.797834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 15:00:57.076635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Because I want to use Persian (Farsi) data to finetune the model, I will use another tokenizer which is for T5 models but trained on Persian datasets. For more info check out [here](https://huggingface.co/Ahmad/parsT5-base)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Ahmad/parsT5-base") # Using Persian (Farsi) tokenizer - our training data is Farsi
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Importing data and preprocess

In [ ]:
!kaggle datasets download -d fatemehmahdibabaee/persian-news
!unzip persian-news.zip -d data

Dataset URL: https://www.kaggle.com/datasets/fatemehmahdibabaee/persian-news
License(s): unknown
100%|███████████████████████████████████████| 90.6M/90.6M [02:25<00:00, 713kB/s]
100%|███████████████████████████████████████| 90.6M/90.6M [02:25<00:00, 654kB/s]
Archive:  persian-news.zip
  inflating: data/pn_summary/dev.csv  
  inflating: data/pn_summary/test.csv  
  inflating: data/pn_summary/train.csv  


In [27]:
data = pd.read_csv('./data/pn_summary/train.csv', delimiter='\t', on_bad_lines='warn')
data.head()

,id,title,article,summary,category,categories,network,link
0,738e296491f8b24c5aa63e9829fd249fb4428a66,مدیریت فروش نفت در دوران تحریم هوشمندانه عمل کرد,به گزارش شانا، علی کاردر امروز (۲۷ دی ماه) در ...,مدیرعامل شرکت ملی نفت، عملکرد مدیریت امور بین‎...,Oil-Energy,نفت,Shana,https://www.shana.ir/news/275284/%D9%85%D8%AF%...
1,00fa692a178a2454419284199df6b6690a75ade0,سبد محصولات پتروشیمی متنوع می‌شود,به گزارش شانا به نقل از شرکت ملی صنایع پتروشیم...,سرپرست مدیریت برنامه‌ریزی و توسعه شرکت ملی صنا...,Oil-Energy,پتروشیمی,Shana,https://www.shana.ir/news/293940/%D8%B3%D8%A8%...
2,1bdb42b53c080b36318b82051edacb5c8f61f6a2,معرفی گوگرد بنتونیتی پالایشگاه خانگیران در نما...,به گزارش شانا به نقل از شرکت پالایش گاز شهید ه...,پالایشگاه گاز خانگیران با هدف معرفی گوگرد بنتو...,Oil-Energy,گاز,Shana,https://www.shana.ir/news/292952/%D9%85%D8%B9%...
3,73ef47636beaf86610695f62716da113624ed315,روند عمرانی شیراز با فروکش کردن کرونا عادی می‌شود,به گزارش خبرنگار ایمنا، سعید نظری در صفحه اینس...,سخنگوی شورای شهر شیراز گفت: روند عمرانی و شهرس...,Local,پارلمان شهری,Imna,https://www.imna.ir/news/416660/%D8%B1%D9%88%D...
4,0c45a2e8b760cb6779a8be426f0075893e4e8b44,قدردانی از اقدام ایثارگرانه نیروی حراست در اطف...,به گزارش شانا، سیدباقر مرتضوی، مشاور وزیر نفت ...,مشاور وزیر نفت و مدیرکل اچ اس یی و پدافند غیرع...,Oil-Energy,گاز,Shana,https://www.shana.ir/news/277191/%D9%82%D8%AF%...


I only need `article` and `summary` columns, So I'll delete rest of the columns

In [28]:
data = data.drop(columns=['id', 'title', 'network', 'link', 'category', 'categories'], axis=0)
data.tail()

,article,summary
82017,به گزارش ایمنا، تیم‌های ملی هاکی زنان و مردان ...,تیم‌های ملی هاکی زنان و مردان ایران در سومین د...
82018,به گزارش بازار، مصطفی قلی خسروی افزود: در کشور...,قلی خسروی، رئیس اتحادیه مشاوران املاک تهران گف...
82019,به گزارش ایمنا، به نقل از پایگاه اطلاع‌رسانی ک...,رئیس کمیته امداد از آغاز مرحله دوم پویش ایران ...
82020,به گزارش خبرگزاری خبرآنلاین و به نقل از ایران ...,گروه صنعتی ایران‌خودرو به منظور تامین نیاز مشت...
82021,به گزارش شانا به نقل از دبیرخانه سازمان کشورها...,مجموع کاهش جهانی تولید نفت خام می‌تواند به بیش...


In [29]:
# This function will normalize the text
def clear_text(text: str) -> str:
    text = re.sub("[^آ-ی۰-۹]+", " ", text)
    return text

In [30]:
# Applying normalize function on data columns
data["article"] = data["article"].apply(clear_text)
data["summary"] = data["summary"].apply(clear_text)

# Show a sample after normalization
data.iloc[0]

article    به گزارش شانا علی کاردر امروز ۲۷ دی ماه در مرا...
summary    مدیرعامل شرکت ملی نفت عملکرد مدیریت امور بین ا...
Name: 0, dtype: object

As our traiing data is so big, I only use 30% of this data for all `train`, `val` and `test` datasets

In [35]:
data = data.sample(frac=0.3) # Using 30% of data

train_data = data.sample(frac=0.7) # train data
test_data = data.drop(train_data.index)

eval_data = test_data.sample(frac=0.5) # eval data
test_data = test_data.drop(eval_data.index) # test data

train_data.shape, eval_data.shape, test_data.shape

((17225, 2), (3691, 2), (3691, 2))

In [37]:
# This function will tokenize our data
def tokenize_data(data):
    start = "Summerize the following article: "
    end = "Summary: "
    prompt = [start + article + end for article in data["article"]]

    data["input_ids"] = tokenizer(prompt, max_length=512, padding='max_length', truncation=True, return_tensors='pt').input_ids
    data["labels"] = tokenizer(data["summary"], max_length=512, padding='max_length', truncation=True, return_tensors='pt').input_ids

    return data

In [38]:
# Converting Pandas obj to Dataset obj so we can send the data directly to the model
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)
test_dataset = Dataset.from_pandas(test_data)

In [39]:
# Tokenizeing data
train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(tokenize_data, batched=True, remove_columns=eval_dataset.column_names)
test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/17225 [00:00<?, ? examples/s]

Map:   0%|          | 0/3691 [00:00<?, ? examples/s]

Map:   0%|          | 0/3691 [00:00<?, ? examples/s]

# Set Trainign args and start trainig

In [40]:
BATCH_SIZE = 4
EPOCHS = 3
MODEL_NAME = "google-flan-t5-base-finetune-summarize-persian-news"
MAX_SAVE = 2
LR = 1e-3

trainign_args = TrainingArguments(
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    save_total_limit=MAX_SAVE,
    output_dir=MODEL_NAME,
    learning_rate=LR,
    evaluation_strategy='epoch',
    report_to="none"

)

trainer = Trainer(
    args=trainign_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()

MODEL_PATH = f"./{MODEL_NAME}/final_model"

trainer.model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.228000,0.209826
2,0.195900,0.191676
3,0.169400,0.188280


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

('./google-flan-t5-base-finetune-summarize-persian-news/final_model/tokenizer_config.json',
 './google-flan-t5-base-finetune-summarize-persian-news/final_model/special_tokens_map.json',
 './google-flan-t5-base-finetune-summarize-persian-news/final_model/tokenizer.json')

# Evaluate and Test finetuned model

In [41]:
# Evaluate fine-tuned model
test_result = trainer.evaluate(eval_dataset=test_dataset)
print(test_result)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.19226735830307007, 'eval_runtime': 268.255, 'eval_samples_per_second': 13.759, 'eval_steps_per_second': 1.722, 'epoch': 3.0}


In [42]:
# Test the new model

TEST_TEXT = """
به گزارش ایرنا به نقل از اداره کل هواشناسی خوزستان در این اطلاعیه آمده است: وزش باد متوسط تا نسبتا شدید همراه با تندباد لحظه ای، رخداد گرد و خاک محلی و همچنین احتمال وقوع گرد و خاک همرفتی از بعد از ظهر امروز امروز دوشنبه تا اوایل وقت فردا سه شنبه در مناطق غرب، جنوب ، جنوب غرب و تا حدودی مرکزی استان قابل انتظار است که سبب کاهش دید افقی و کیفیت هوا و افزایش غلظت آلاینده‌های گرد و خاک خواهد شد.
مدیر کل هواشناسی خوزستان گفت: براساس آخرین نقشه‌های پیش‌یابی هواشناسی، تا اواسط روز چهارشنبه جریانات جنوبی سبب افزایش رطوبت و شرجی در مناطق ساحلی، جنوبی، غربی و مرکزی استان خواهد شد.
محمد سبزه زاری افزود: طی امروز و فردا عبور موج ضعیف بارشی و بالابودن ضرایب ناپایداری سبب رشد ابر در بیشتر مناطق استان و رگبارهای پراکنده همراه با رعد وبرق و وزش باد متوسط در مناطق شمالی و شرقی خواهد شد.
"""

start = "Summerize the following article: "
end = "Summary: "
prompt = start + TEST_TEXT + end

In [43]:
from transformers import GenerationConfig
device = "cuda" if torch.cuda.is_available() else "cpu"
trained_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
tokenized_test_text = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
test_output = trained_model.generate(tokenized_test_text, generation_config=GenerationConfig(max_new_tokens=200))[0]

In [45]:
result = tokenizer.decode(test_output, skip_special_tokens=True)
result

'مدیر کل هواشناسی خوزستان گفت وزش باد متوسط تا نسبتا شدید همراه با تندباد لحظه ایی و خاک محلی و همچنین احتمال وقوع گرد و خاک همرفتی از بعد از ظهر امروز دوشنبه تا اوایل وقت فردا سه شنبه در مناطق غربی جنوب غرب و تا حدودی مرکزی استان قابل انتظار است که سبب کاهش دید افقی و کیفیت هوا و افزایش غلظت آلاینده های گرد و خاک خواهد شد '

In [46]:
from huggingface_hub import notebook_login
notebook_login()

In [49]:
# Push to 🤗 hub
trainer.push_to_hub("ali619/google-flan-t5-base-finetune-summarize-persian-news")

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ali619/checkpoints_training/commit/7028dc5c0e1b86ac8220def80dafd7bb23ab4962', commit_message='ali619/google-flan-t5-base-finetune-summarize-persian-news', commit_description='', oid='7028dc5c0e1b86ac8220def80dafd7bb23ab4962', pr_url=None, pr_revision=None, pr_num=None)